In [32]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle
import os
import random

class FantasyFootballDraftAssistant:
    def __init__(self):
        self.df = None
        self.positions = {'QB': (1, 2), 'RB': (6, 9), 'WR': (5, 9), 'TE': (1, 2), 'K': (1, 1), 'DST': (1, 1)}
        self.starting_positions = {'QB': 1, 'RB': 2, 'WR': 3, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
        self.flex_positions = ['RB', 'WR', 'TE']
        self.q_table = {}
        self.alpha = 0.1  # Learning rate
        self.gamma = 0.9  # Discount factor
        self.epsilon = 0.1  # Exploration rate
        self.total_episodes = 0
        self.base_exploration_N = 15  # Maximum exploration cap
        self.min_exploration_N = 5    # Minimum exploration, even in late rounds
        self.position_exploration_factor = {
            'QB': 0.7, 'RB': 1.2, 'WR': 1.2, 'TE': 0.9, 'K': 0.3, 'DST': 0.3
        }

        self.epsilon_start = 0.2  # Starting exploration rate
        self.epsilon_end = 0.01  # Ending exploration rate
        self.epsilon_decay = 0.9999  # Decay rate for epsilon
        self.epsilon = self.epsilon_start
        self.num_teams = 12

    def get_exploration_N(self, round_num, position):
        # Dynamic round factor: starts high, decreases as draft progresses
        round_factor = max(0.5, 2 - (round_num - 1) * 0.1)  # Starts at 2, decreases by 0.1 each round, min 0.5
        
        # Apply position factor
        position_factor = self.position_exploration_factor.get(position, 1.0)
        
        # Calculate exploration N
        exploration_N = int(self.base_exploration_N * round_factor * position_factor)
        
        # Ensure N is within our defined range
        return max(self.min_exploration_N, min(exploration_N, self.base_exploration_N))

    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_end, self.epsilon * self.epsilon_decay)  
        
    def load_data(self, file_path):
        self.df = pd.read_csv(file_path)
        self.df['ADP'] = pd.to_numeric(self.df['ADP'], errors='coerce')
        self.df = self.df.sort_values('ADP').reset_index(drop=True)
        self.players = self.df.to_dict('records')
        
    def get_state(self, team, round_num, pick_number):
        pos_counts = [sum(1 for p in team if p['pos'] == pos) for pos in self.positions.keys()]
        return tuple(pos_counts + [round_num, pick_number])
    
    def get_actions(self, available_players, team, round_num):
        actions = []
        for player in available_players:
            pos = player['pos']
            pos_count = sum(1 for p in team if p['pos'] == pos)
            if pos_count < self.positions[pos][1] and (pos not in ['K', 'DST'] or round_num > 12):
                actions.append(player['player'])
        return actions
    
    def calculate_team_value(self, team):
        weekly_scores = []
        for week in range(1, 18):  # 17-week season
            available_players = [p for p in team if p['bye_week'] != week]
            starters = self.select_starters(available_players)
            week_score = sum(p['ppr_projection'] / 17 for p in starters)
            weekly_scores.append(week_score)
        
        total_score = sum(weekly_scores)
        bench_strength = sum(p['ppr_projection'] for p in team) - sum(p['ppr_projection'] for p in self.select_starters(team))
        return total_score + (bench_strength * 0.1)  # Adding some value for bench strength
    
    def select_starters(self, available_players):
        starters = []
        for pos, count in self.starting_positions.items():
            if pos == 'FLEX':
                flex_options = sorted([p for p in available_players if p['pos'] in self.flex_positions and p not in starters], 
                                      key=lambda x: x['ppr_projection'], reverse=True)
                starters.extend(flex_options[:count])
            else:
                pos_players = sorted([p for p in available_players if p['pos'] == pos and p not in starters], 
                                     key=lambda x: x['ppr_projection'], reverse=True)
                starters.extend(pos_players[:count])
        return starters
    
    def calculate_reward(self, team, player, round_num):
        team_value_before = self.calculate_team_value(team)
        new_team = team + [player]
        team_value_after = self.calculate_team_value(new_team)
        value_added = team_value_after - team_value_before
        
        adp_bonus = max(0, (200 - player['ADP']) / 10) if not np.isnan(player['ADP']) else 0
        
        pos_counts = {pos: sum(1 for p in team if p['pos'] == pos) for pos in self.positions.keys()}
        
        # Prioritize key positions in early rounds
        if round_num <= 6:
            if player['pos'] == 'QB':
                if pos_counts['QB'] == 0:
                    value_added *= 1.5
                else:
                    value_added *= 0.5  # Penalize drafting a second QB early
            elif player['pos'] in ['RB', 'WR']:
                value_added *= 1.5
            elif player['pos'] == 'TE':
                if pos_counts['TE'] == 0:
                    value_added *= 1.5
                else:
                    value_added *= 0.5  # Penalize drafting a second TE early
        
        # Penalize drafting K and DST early
        if player['pos'] in ['K', 'DST'] and round_num <= 12:
            value_added *= 0.1
        
        # Boost value for filling starting lineup
        if pos_counts[player['pos']] < self.starting_positions.get(player['pos'], 0):
            value_added *= 1.5
        
        return value_added + adp_bonus
    
    def train(self, num_episodes=10000):
        start_episode = self.total_episodes
        for episode in range(start_episode, start_episode + num_episodes):
            if episode % 200 == 0:
                print(f"Training episode {episode}/{start_episode + num_episodes}")
            
            user_position = random.randint(0, self.num_teams - 1)  # Randomize user position for each episode
            teams = [[] for _ in range(self.num_teams)]
            available_players = self.players.copy()
            
            for round_num in range(1, 19):
                draft_order = range(self.num_teams) if round_num % 2 == 1 else reversed(range(self.num_teams))
                for pick_in_round, team_index in enumerate(draft_order):
                    pick_number = (round_num - 1) * self.num_teams + pick_in_round + 1
                    state = self.get_state(teams[team_index], round_num, pick_number)
                    actions = self.get_actions(available_players, teams[team_index], round_num)
                    
                    if not actions:
                        continue
                    
                    if np.random.uniform(0, 1) < self.epsilon:
                        # Exploration: Choose randomly from top N ADP players
                        sorted_actions = sorted(actions, key=lambda a: next((p['ADP'] for p in available_players if p['player'] == a), float('inf')))
                        player = next(p for p in available_players if p['player'] == sorted_actions[0])
                        position = player['pos']
                        exploration_N = self.get_exploration_N(round_num, position)
                        top_N_adp = sorted_actions[:exploration_N]
                        action = np.random.choice(top_N_adp)
                    else:
                        # Exploitation: Choose the action with the highest Q-value
                        q_values = [self.q_table.get((state, a), 0) for a in actions]
                        action = actions[np.argmax(q_values)]
                    
                    player = next(p for p in available_players if p['player'] == action)
                    reward = self.calculate_reward(teams[team_index], player, round_num)
                    teams[team_index].append(player)
                    available_players = [p for p in available_players if p['player'] != action]
                    
                    next_round = round_num + 1 if pick_in_round == self.num_teams - 1 else round_num
                    next_pick = (pick_number % self.num_teams) + 1
                    next_state = self.get_state(teams[team_index], next_round, next_pick)
                    
                    # Q-value update
                    max_future_q = max([self.q_table.get((next_state, a), 0) for a in self.get_actions(available_players, teams[team_index], next_round)], default=0)
                    current_q = self.q_table.get((state, action), 0)
                    new_q = (1 - self.alpha) * current_q + self.alpha * (reward + self.gamma * max_future_q)
                    self.q_table[(state, action)] = new_q
            
            # Decay epsilon after each episode
            self.decay_epsilon()
        
        self.total_episodes += num_episodes
    
    def recommend_players(self, team, available_players, round_num,pick_number, num_recommendations=5):
        state = self.get_state(team, round_num, pick_number)
        actions = self.get_actions(available_players, team, round_num)
        q_values = [(a, self.q_table.get((state, a), 0)) for a in actions]
        top_actions = sorted(q_values, key=lambda x: x[1], reverse=True)[:num_recommendations]
        player_dict = {p['player']: p for p in available_players}
        return [(player_dict[a], q) for a, q in top_actions]
    
    def simulate_draft(self, user_position, num_teams=12):
        teams = [[] for _ in range(num_teams)]
        available_players = self.players.copy()
        user_picks = []
        
        for round_num in range(1, 19):
            draft_order = range(self.num_teams) if round_num % 2 == 1 else reversed(range(self.num_teams))
            for pick_in_round, team_index in enumerate(draft_order):
                pick_number = (round_num - 1) * self.num_teams + pick_in_round + 1
                if team_index == user_position:
                    recommendations = self.recommend_players(teams[team_index], available_players, round_num, pick_number)
                    user_picks.append((pick_number, recommendations))
                    
                    if recommendations:
                        selected_player, _ = recommendations[0]
                        teams[team_index].append(selected_player)
                        available_players = [p for p in available_players if p['player'] != selected_player['player']]
                else:
                    # Other teams draft based on ADP with round-based randomness
                    valid_players = [p for p in available_players if sum(1 for player in teams[team_index] if player['pos'] == p['pos']) < self.positions[p['pos']][1]]
                    if valid_players:
                        # Sort valid players by ADP, handling NaN values
                        sorted_players = sorted(valid_players, key=lambda p: p['ADP'] if not np.isnan(p['ADP']) else float('inf'))
                        
                        # Determine the number of top players to consider based on the round
                        if round_num <= 1:
                            top_n = 2  # Pick the top ADP in rounds 1 and 2
                        elif round_num <= 3:
                            top_n = 2  # Pick from top 2 ADP in round 3
                        elif 4 <= round_num <= 6:
                            top_n = 3  # Pick from top 3 ADP in rounds 4, 5, 6
                        else:
                            top_n = 5  # Pick from top 5 ADP in later rounds
                        
                        # Select top N players (or fewer if less than N are available)
                        top_players = sorted_players[:min(top_n, len(sorted_players))]
                        
                        # Randomly select one player from the top players
                        player = random.choice(top_players)
                        teams[team_index].append(player)
                        available_players = [p for p in available_players if p['player'] != player['player']]
        
        return user_picks, teams
    
    def save_model(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump((self.q_table, self.total_episodes), f)
        print(f"Model saved to {file_path}")
    
    def load_model(self, file_path):
        with open(file_path, 'rb') as f:
            self.q_table, self.total_episodes = pickle.load(f)
        print(f"Model loaded from {file_path}. Total episodes: {self.total_episodes}")

In [33]:
# Usage and testing
assistant = FantasyFootballDraftAssistant()
assistant.load_data('data//cbs_fantasy_projection_master.csv')

model_file = 'models//fantasy_football_model.pkl'

# Check if a saved model exists
if os.path.exists(model_file):
    assistant.load_model(model_file)
    print("Continuing training from saved model...")
else:
    print("Starting new training...")

# Train for 10,000 episodes
print("Training the model...")
assistant.train(num_episodes=50000)

# Save the model after training
assistant.save_model(model_file)


Model loaded from models//fantasy_football_model.pkl. Total episodes: 60000
Continuing training from saved model...
Training the model...
Training episode 60000/110000
Training episode 60200/110000
Training episode 60400/110000
Training episode 60600/110000
Training episode 60800/110000
Training episode 61000/110000
Training episode 61200/110000
Training episode 61400/110000
Training episode 61600/110000
Training episode 61800/110000
Training episode 62000/110000
Training episode 62200/110000
Training episode 62400/110000
Training episode 62600/110000
Training episode 62800/110000
Training episode 63000/110000
Training episode 63200/110000
Training episode 63400/110000
Training episode 63600/110000
Training episode 63800/110000
Training episode 64000/110000
Training episode 64200/110000
Training episode 64400/110000
Training episode 64600/110000
Training episode 64800/110000
Training episode 65000/110000
Training episode 65200/110000
Training episode 65400/110000
Training episode 65600

In [34]:


user_position=7
print(f"\nSimulating a draft for a user in position {user_position + 1}...")
user_picks, simulated_teams = assistant.simulate_draft(user_position=user_position)  # 0-based index

print("\nRecommendations for your picks:")
for pick_number, recommendations in user_picks:
    print(f"\nPick {pick_number}:")
    for i, (player, q_value) in enumerate(recommendations, 1):
        print(f"  {i}. {player['player']} ({player['pos']}) - Q-value: {q_value:.2f}, ADP: {player['ADP']:.2f}")

print("\nYour team:")
for player in simulated_teams[user_position]:  # user_position is 2 (0-based index)
    print(f"{player['player']} ({player['pos']}) - ADP: {player['ADP']:.2f}, Projection: {player['ppr_projection']:.2f}")

# Analyze position distribution for all teams
def analyze_team(team):
    positions = {pos: 0 for pos in assistant.positions.keys()}
    for player in team:
        positions[player['pos']] += 1
    return positions

print("\nPosition distribution for all teams:")
for i, team in enumerate(simulated_teams):
    positions = analyze_team(team)
    print(f"Team {i+1}: {positions}")

# Calculate and print total projected points for each team
print("\nTotal projected points for each team:")
for i, team in enumerate(simulated_teams):
    total_points = sum(player['ppr_projection'] for player in team)
    print(f"Team {i+1}: {total_points:.2f}")


Simulating a draft for a user in position 8...

Recommendations for your picks:

Pick 8:
  1. A.J. Brown (WR) - Q-value: 55.45, ADP: 10.20
  2. Puka Nacua (WR) - Q-value: 55.40, ADP: 12.40
  3. Bijan Robinson (RB) - Q-value: 54.60, ADP: 6.00
  4. Jonathan Taylor (RB) - Q-value: 54.52, ADP: 11.20
  5. Marvin Harrison Jr. (WR) - Q-value: 52.93, ADP: 18.00

Pick 17:
  1. Jalen Hurts (QB) - Q-value: 73.89, ADP: 31.80
  2. Travis Kelce (TE) - Q-value: 51.06, ADP: 23.40
  3. Davante Adams (WR) - Q-value: 50.54, ADP: 18.40
  4. Travis Etienne (RB) - Q-value: 50.09, ADP: 18.20
  5. Derrick Henry (RB) - Q-value: 49.60, ADP: 18.80

Pick 32:
  1. DeVonta Smith (WR) - Q-value: 46.65, ADP: 43.40
  2. Stefon Diggs (WR) - Q-value: 42.53, ADP: 37.80
  3. DJ Moore (WR) - Q-value: 42.30, ADP: 43.00
  4. DK Metcalf (WR) - Q-value: 42.01, ADP: 42.20
  5. Jaylen Waddle (WR) - Q-value: 41.83, ADP: 40.80

Pick 41:
  1. Zay Flowers (WR) - Q-value: 42.46, ADP: 57.80
  2. Trey McBride (TE) - Q-value: 40.61, AD

In [19]:


user_position=3
print(f"\nSimulating a draft for a user in position {user_position}...")
user_picks, simulated_teams = assistant.simulate_draft(user_position=user_position)  # 0-based index

print("\nRecommendations for your picks:")
for pick_number, recommendations in user_picks:
    print(f"\nPick {pick_number}:")
    for i, (player, q_value) in enumerate(recommendations, 1):
        print(f"  {i}. {player['player']} ({player['pos']}) - Q-value: {q_value:.2f}, ADP: {player['ADP']:.2f}")

print("\nYour team:")
for player in simulated_teams[user_position]:  # user_position is 2 (0-based index)
    print(f"{player['player']} ({player['pos']}) - ADP: {player['ADP']:.2f}, Projection: {player['ppr_projection']:.2f}")

# Analyze position distribution for all teams
def analyze_team(team):
    positions = {pos: 0 for pos in assistant.positions.keys()}
    for player in team:
        positions[player['pos']] += 1
    return positions

print("\nPosition distribution for all teams:")
for i, team in enumerate(simulated_teams):
    positions = analyze_team(team)
    print(f"Team {i+1}: {positions}")

# Calculate and print total projected points for each team
print("\nTotal projected points for each team:")
for i, team in enumerate(simulated_teams):
    total_points = sum(player['ppr_projection'] for player in team)
    print(f"Team {i+1}: {total_points:.2f}")


Simulating a draft for a user in position 3...

Recommendations for your picks:

Pick 4:
  1. Christian McCaffrey (RB) - Q-value: 332.64, ADP: 1.00
  2. Saquon Barkley (RB) - Q-value: 327.68, ADP: 10.40
  3. Bijan Robinson (RB) - Q-value: 327.02, ADP: 6.00
  4. Amon-Ra St. Brown (WR) - Q-value: 321.46, ADP: 6.80
  5. Jonathan Taylor (RB) - Q-value: 320.97, ADP: 11.20

Pick 21:
  1. Jalen Hurts (QB) - Q-value: 291.61, ADP: 31.80
  2. Josh Allen (QB) - Q-value: 289.89, ADP: 26.20
  3. Patrick Mahomes (QB) - Q-value: 285.09, ADP: 29.60
  4. Lamar Jackson (QB) - Q-value: 283.11, ADP: 37.60
  5. Chris Olave (WR) - Q-value: 283.03, ADP: 23.40

Pick 28:
  1. Cooper Kupp (WR) - Q-value: 241.04, ADP: 38.20
  2. Mike Evans (WR) - Q-value: 240.71, ADP: 30.80
  3. Brandon Aiyuk (WR) - Q-value: 240.35, ADP: 32.80
  4. DeVonta Smith (WR) - Q-value: 239.97, ADP: 43.40
  5. Nico Collins (WR) - Q-value: 239.75, ADP: 32.80

Pick 45:
  1. DJ Moore (WR) - Q-value: 214.51, ADP: 43.00
  2. DK Metcalf (WR) 